### Question 1 : Part 1

In [1]:
import pandas as pd
import warnings
import time
import csv
import heapq
from geopy.distance import geodesic as GD
from collections import defaultdict 
warnings.filterwarnings('ignore')

In [2]:
df_all = pd.read_csv("nyc_taxi_trips/nyc_taxi_data.csv")
df_all.head()

,Unnamed: 0,tpep_pickup_datetime,tpep_dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance,pickup_hrs,dropoff_hrs,day_week,tpep_pickup_timestamp,tpep_dropoff_timestamp,duration,speed
0,0,2015-01-24 18:00:28,2015-01-24 18:10:07,-73.964111,40.761398,-73.977989,40.783093,2.40,18,18,5,1422122428,1422123007,579,14
1,1,2015-01-15 4:37:29,2015-01-15 4:58:14,-73.961479,40.760410,-73.943573,40.709702,5.00,4,4,3,1421296649,1421297894,1245,14
2,2,2015-01-09 5:14:31,2015-01-09 5:47:16,-73.986893,40.761726,-73.873169,40.774326,10.25,5,5,4,1420780471,1420782436,1965,18
3,3,2015-01-29 9:52:05,2015-01-29 10:16:33,-73.965759,40.758114,-74.010399,40.701965,5.46,9,10,3,1422525125,1422526593,1468,13
4,4,2015-01-02 2:20:01,2015-01-02 2:27:37,-73.955032,40.821857,-73.950897,40.808399,1.50,2,2,4,1420165201,1420165657,456,11


In [3]:
df = df_all[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','trip_distance']]
df.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance
0,-73.964111,40.761398,-73.977989,40.783093,2.40
1,-73.961479,40.760410,-73.943573,40.709702,5.00
2,-73.986893,40.761726,-73.873169,40.774326,10.25
3,-73.965759,40.758114,-74.010399,40.701965,5.46
4,-73.955032,40.821857,-73.950897,40.808399,1.50


In [4]:
# lat_long = df_all.iloc[:,3:8]
lat_long = ['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'] 
for column in lat_long:
    df[column] = df[column].apply(lambda x : round(x,4)) 

In [5]:
# graphs list
subset_from_pickup = df[['pickup_longitude','pickup_latitude']].to_numpy()
subset_from_dropoff = df[['dropoff_longitude','dropoff_latitude']].to_numpy()

location_set = set()
graph_nodes = dict()

for location in subset_from_pickup:
    location_set.add(tuple(location))
    
for loc in subset_from_dropoff:
    location_set.add(tuple(loc))
    
for index, node in enumerate(location_set):
    graph_nodes[node] = index
    
nodes_df = pd.DataFrame([(v, k[0], k[1]) for k, v in graph_nodes.items()], columns=['nodeid', 'lat', 'long'])
nodes_df.to_csv("nyc_taxi_trips/nodes.csv")


### Question 1 : Part 2

In [6]:
tuples_list1 = [tuple(x) for x in subset_from_pickup]
tuples_list2 = [tuple(x) for x in subset_from_dropoff]
cost = df[["trip_distance"]]

edges = []
for i in range(len(tuples_list1)):
    edges.append([graph_nodes[tuples_list1[i]], graph_nodes[tuples_list2[i]]]) 


In [7]:
edges_df = pd.DataFrame(edges, columns = ['nodeid1', 'nodeid2']) 
edges_df.to_csv("nyc_taxi_trips/edges.csv")

In [8]:
weighted_list = pd.concat([edges_df, cost], axis=1).to_numpy()
weighted_list

array([[4.8293e+04, 2.8440e+04, 2.4000e+00],
       [4.5189e+04, 6.6770e+03, 5.0000e+00],
       [6.1317e+04, 6.0751e+04, 1.0250e+01],
       ...,
       [6.4343e+04, 8.8601e+04, 1.0000e+00],
       [5.1248e+04, 1.0473e+04, 2.5800e+00],
       [5.0716e+04, 2.4589e+04, 3.0700e+01]])

In [9]:
def create_graph(weighted_list):
    graph = defaultdict(dict)
    for src,dest,cost in weighted_list:
        graph[int(src)][int(dest)]=cost
    return graph

graph = create_graph(weighted_list)

In [10]:
def helper(graph, src):
    queue = [src]
    min_cost = {k : float('inf') for k in graph}
    min_cost[src] = 0
    previous_nodes = {}
    
    while queue:
        current_node = queue.pop(0)
        for node in graph[current_node]:
            if node in min_cost:
                new_cost = min_cost[current_node] + graph[current_node][node]
                if new_cost < min_cost[node]:
                    min_cost[node] = min(new_cost, min_cost[node])
                    queue.append(node)
                    previous_nodes[node] = current_node
    return min_cost, previous_nodes


In [11]:
def UCS(graph, src, dest):
    time1 = time.time()
    minDistances, predecessor = helper(graph, src)
    
    path = []
    currentNode = dest
    while currentNode != src:
        if currentNode not in predecessor:
            print("Path does not exist")
            break
        else:
            path.insert(0, currentNode)
            currentNode = predecessor[currentNode]
    path.insert(0, src)
    
    if dest in minDistances and minDistances[dest] != float("inf"):
        print('Cost ' + str(minDistances[dest]))
        print('Path ' + str(path))
    time2 = time.time()
    print("Time taken by Uniform Cost Search is " +str(round(time2-time1, 4)) +" sec")


In [12]:
src = 48293
dest = 38931
min_cost, previous_nodes = helper(graph, src)
UCS(graph, src, dest)

Cost 0.82
Path [48293, 38931]
Time taken by Uniform Cost Search is 0.0108 sec


### Question 1 : Part 3

In [13]:
def get_heuristics(pickup_locations, dropoff_locations):
    heuristics= []
    for i in range(len(pickup_locations)):
        distance =  GD(pickup_locations[i], dropoff_locations[i]).km 
        heuristics.append(distance)
    return heuristics

heuristics = get_heuristics(tuples_list1, tuples_list2)

In [14]:
def a_star_graph(weighted_list, heuristics):
    graph = defaultdict(dict)
    for i in range(len(weighted_list)):
        src = weighted_list[i][0]
        dest = weighted_list[i][1]
        cost = weighted_list[i][2]
        graph[int(src)][int(dest)] = [cost, heuristics[i]] 
    return graph 

graph_a_star = a_star_graph(weighted_list, heuristics)

In [15]:
def astar(graph,start_node,end_node):
    
    time1 = time.time()
    f_distance = {node:float('inf') for node in graph}
    f_distance[start_node] = 0
    
    g_distance = {node:float('inf') for node in graph}
    g_distance[start_node] = 0
    
    came_from = {node:None for node in graph}
    came_from[start_node] = start_node
    
    queue = [(0,start_node)]
    path = []
    
    while queue:
        current_f_distance, current_node = heapq.heappop(queue)

        if current_node == end_node:
            time2 = time.time()
            time_taken = time2-time1
            return current_f_distance, path, time_taken
        
        for next_node,weights in graph[current_node].items():
            temp_g_distance = g_distance[current_node] + weights[0]
            if next_node in g_distance:
                if temp_g_distance < g_distance[next_node]:
                    g_distance[next_node] = temp_g_distance
                    heuristic = weights[1]
                    f_distance[next_node] = temp_g_distance + heuristic
                    came_from[next_node] = current_node
                    path.append(next_node)

                    heapq.heappush(queue,(f_distance[next_node],next_node))
    time2 = time.time()
    time_taken = time2-time1
    return current_f_distance, path, time_taken

In [16]:
src = 48293
dest = 38931
cost, path, time_taken = astar(graph_a_star, src, dest)
print("Path "+str(path)) 
print("Cost "+str(round(cost,4)))
print("Time taken by A* search " +str(round(time_taken,4)) +" sec")

Path [28440, 38931, 57087]
Cost 1.8437
Time taken by A* search 0.0263 sec
